In [1]:
!pip install --upgrade pip
!pip install gdown

     |████████████████████████████████| 2.1 MB 891 kB/s 
  Attempting uninstall: pip
    Found existing installation: pip 20.1.1
    Uninstalling pip-20.1.1:
      Successfully uninstalled pip-20.1.1
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - \ | / - done
  Preparing metadata (pyproject.toml) ... - \ | / - done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14775 sha256=d46ee4dc09022040a8d26dce5c6c7af16163a98b75225a2c419f7900816e3e10
  Stored in directory: /root/.cache/pip/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown


In [2]:
from keras.models import load_model

In [3]:
import os
if not (os.path.isfile('data.zip') and os.path.isfile('data.zip')):
    !rm -rf model_discovery
    import gdown
    gdown.download('https://drive.google.com/uc?id=1exw9vOYcb0fPVseleifK0m1VxbBCsBM_', output=None, quiet=False)
    !unzip -qq model_discovery.zip

Downloading...
From: https://drive.google.com/uc?id=1exw9vOYcb0fPVseleifK0m1VxbBCsBM_
To: /kaggle/working/model_discovery.zip
100%|██████████| 11.9k/11.9k [00:00<00:00, 4.07MB/s]


In [4]:
# import deepmatcher as dm
import itertools
import os
import pandas as pd
import model_discovery
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

from model_discovery import utils, l2lsh
import re

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet


import keras
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras 
from keras import backend as K
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from keras.layers import LSTM,Dense,Bidirectional,Input
from keras.models import Model
import torch
import transformers

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [5]:
datasets = ['../input/imbd-data-1/Movie_1.csv', '../input/imbd-data-1/Movie_2.csv', 
            '../input/financial-news-data-2/Financial_News_2.csv', '../input/financial-news-data-2/Financial_News_1.csv',
            '../input/tweets-data-4/Tweets_1.csv','../input/tweets-data-4/Tweets_2.csv',
           '../input/tweets-hatred-data-5/Tweets_hatred_5.csv']

In [6]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop and i.strip().lower().isalpha():
            final_text.append(i.strip().lower())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text

def fast_encode(texts, tokenizer, chunk_size=256, maxlen=400):

    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(max_length=maxlen)
    all_ids = []
#     print(texts)
    for i in range(0, len(texts), chunk_size):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)



In [7]:
def build_model(transformer, max_len=400):
    
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=2e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

from tokenizers import BertWordPieceTokenizer
# First load the real tokenizer
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased' , lower = True)
# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=True)
bert_model = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased')

In [8]:
comb = list(itertools.permutations(datasets, 2))
print("Source,                                                    Target")
for c in comb:
    print(c[0],"                 ",c[1])

Source,                                                    Target
../input/imbd-data-1/Movie_1.csv                   ../input/imbd-data-1/Movie_2.csv
../input/imbd-data-1/Movie_1.csv                   ../input/financial-news-data-2/Financial_News_2.csv
../input/imbd-data-1/Movie_1.csv                   ../input/financial-news-data-2/Financial_News_1.csv
../input/imbd-data-1/Movie_1.csv                   ../input/tweets-data-4/Tweets_1.csv
../input/imbd-data-1/Movie_1.csv                   ../input/tweets-data-4/Tweets_2.csv
../input/imbd-data-1/Movie_1.csv                   ../input/tweets-hatred-data-5/Tweets_hatred_5.csv
../input/imbd-data-1/Movie_2.csv                   ../input/imbd-data-1/Movie_1.csv
../input/imbd-data-1/Movie_2.csv                   ../input/financial-news-data-2/Financial_News_2.csv
../input/imbd-data-1/Movie_2.csv                   ../input/financial-news-data-2/Financial_News_1.csv
../input/imbd-data-1/Movie_2.csv                   ../input/tweets-data-4/Tweet

In [9]:
arr_model_already_trained=[]
def model_serving(src, tar):
    src_name=src
    tar_name=tar
    src=pd.read_csv(src)
    tar=pd.read_csv(tar)
    
    src['text']=src['text'].apply(denoise_text)
    tar['text']=tar['text'].apply(denoise_text)
        
    #---------------------------------------------------------------------------
    
    data1 = utils.flatten_df(src['text'])
    word1 = utils.word_tokenize(data1)
    data2 = utils.flatten_df(tar['text'])
    word2 = utils.word_tokenize(data2)
    
    jsd=utils.jsd_for_word(word1, word2)
    ada=utils.adaptivity_word(src['text'], tar['text'], word1, word2, threshold=0.6)
    l2d=utils.l2d_btw_domains(word1, word2)
    
    #----------------------------------------------------------------------------
    
    x_train_src,x_test_src,y_train_src,y_test_src = train_test_split(src.text,src.label,random_state = 0 , stratify = src.label)
    x_train_tar,x_test_tar,y_train_tar,y_test_tar = train_test_split(tar.text,tar.label,random_state = 0 , stratify = tar.label)

    x_train_src = fast_encode(x_train_src.values, fast_tokenizer, maxlen=400)
    x_test_src = fast_encode(x_test_src.values, fast_tokenizer, maxlen=400)
    
    x_train_tar = fast_encode(x_train_tar.values, fast_tokenizer, maxlen=400)
    x_test_tar = fast_encode(x_test_tar.values, fast_tokenizer, maxlen=400)

    # Model Save
    filename = re.search('input/(.*).csv', src_name).group(1).split('/')[1]+"_trained_model"
    model_src = build_model(bert_model, max_len=400)
    if filename in arr_model_already_trained: # model saved
        print("Already present-",filename)
        model_src.load_weights(filename)

        print("Loaded")
    else: # model not saved
        print("New model-",filename)
        
        model_src.fit(x_train_src,y_train_src,batch_size = 32 ,validation_data=(x_test_src,y_test_src),epochs = 3)
        arr_model_already_trained.append(filename)
        model_src.save_weights(filename)
        print("Weights saved")
        
    src_acc = model_src.evaluate(x_test_src,y_test_src, verbose=1)[1]*100
    print("src_acc-",src_acc)
    
    ser_acc = model_src.evaluate(x_train_tar,y_train_tar, verbose=1)[1]*100
    print("ser_acc-",ser_acc)
    
    model_tar = build_model(bert_model, max_len=400)
    model_tar.fit(x_train_tar,y_train_tar,batch_size = 32, epochs = 3)
    ft_acc = model_src.evaluate(x_test_tar,y_test_tar, verbose=1)[1]*100
    print("ft_acc-",ft_acc)

    return [jsd,ada,l2d,src_acc, ser_acc, ft_acc]

In [10]:
comb = list(itertools.permutations(datasets, 2))
src_list = []
tar_list = []
src_acc_list = []
ser_acc_list = []
ft_acc_list = []
jsd_list = []
ada_list = []
l2d_list = []
for src, tar in tqdm(comb, leave=False):
    jsd, ada, l2d, src_acc, ser_acc, ft_acc = model_serving(src, tar)
    jsd_list.append(jsd)
    ada_list.append(ada)
    src=re.search('input/(.*).csv', src).group(1).split('/')[1]
    tar=re.search('input/(.*).csv', tar).group(1).split('/')[1]
    src_list.append(src)
    tar_list.append(tar)
    l2d_list.append(l2d)
    src_acc_list.append(src_acc)
    ser_acc_list.append(ser_acc)
    ft_acc_list.append(ft_acc)

New model- Movie_1_trained_model
Epoch 1/3
118/118 [==============================] - 102s 863ms/step - loss: 0.6180 - accuracy: 0.6264 - val_loss: 0.4145 - val_accuracy: 0.8200
Epoch 2/3
118/118 [==============================] - 100s 850ms/step - loss: 0.3464 - accuracy: 0.8493 - val_loss: 0.3816 - val_accuracy: 0.8328
Epoch 3/3
118/118 [==============================] - 100s 850ms/step - loss: 0.2005 - accuracy: 0.9200 - val_loss: 0.3900 - val_accuracy: 0.8432
Weights saved
40/40 [==============================] - 10s 240ms/step - loss: 0.3900 - accuracy: 0.8432
src_acc- 84.32000279426575
118/118 [==============================] - 29s 248ms/step - loss: 0.3573 - accuracy: 0.8539
ser_acc- 85.3866696357727
Epoch 1/3
118/118 [==============================] - 91s 769ms/step - loss: 0.3551 - accuracy: 0.8469
Epoch 2/3
118/118 [==============================] - 91s 772ms/step - loss: 0.2234 - accuracy: 0.9099
Epoch 3/3
40/40 [==============================] - 10s 245ms/step - loss: 0.443

/kaggle/working/model_discovery/utils.py:356: RuntimeWarning: invalid value encountered in true_divide
  prob1 = prob1 / np.sum(prob1)


New model- Financial_News_2_trained_model
Epoch 1/3
24/24 [==============================] - 21s 859ms/step - loss: 0.5886 - accuracy: 0.7087 - val_loss: 0.3847 - val_accuracy: 0.8333
Epoch 2/3
24/24 [==============================] - 19s 802ms/step - loss: 0.2761 - accuracy: 0.8943 - val_loss: 0.2736 - val_accuracy: 0.8862
Epoch 3/3
24/24 [==============================] - 19s 799ms/step - loss: 0.1625 - accuracy: 0.9472 - val_loss: 0.2694 - val_accuracy: 0.8862
Weights saved
8/8 [==============================] - 2s 210ms/step - loss: 0.2694 - accuracy: 0.8862
src_acc- 88.61788511276245
118/118 [==============================] - 29s 249ms/step - loss: 0.1477 - accuracy: 0.9475
ser_acc- 94.74666714668274
Epoch 1/3
118/118 [==============================] - 91s 770ms/step - loss: 0.1747 - accuracy: 0.9269
Epoch 2/3
118/118 [==============================] - 91s 770ms/step - loss: 0.0658 - accuracy: 0.9805
Epoch 3/3
40/40 [==============================] - 10s 238ms/step - loss: 0.3535 

/kaggle/working/model_discovery/utils.py:356: RuntimeWarning: invalid value encountered in true_divide
  prob1 = prob1 / np.sum(prob1)


Already present- Financial_News_2_trained_model
Loaded
8/8 [==============================] - 2s 211ms/step - loss: 0.2694 - accuracy: 0.8862
src_acc- 88.61788511276245
118/118 [==============================] - 29s 247ms/step - loss: 0.1116 - accuracy: 0.9608
ser_acc- 96.07999920845032
Epoch 1/3
118/118 [==============================] - 91s 770ms/step - loss: 0.1275 - accuracy: 0.9520
Epoch 2/3
118/118 [==============================] - 91s 770ms/step - loss: 0.0456 - accuracy: 0.9829
Epoch 3/3
40/40 [==============================] - 10s 241ms/step - loss: 0.4007 - accuracy: 0.8400
ft_acc- 83.99999737739563


...

Already present- Financial_News_1_trained_model
Loaded
8/8 [==============================] - 2s 212ms/step - loss: 0.3462 - accuracy: 0.8780
src_acc- 87.80487775802612
118/118 [==============================] - 29s 248ms/step - loss: 0.8872 - accuracy: 0.6360
ser_acc- 63.599997758865356
Epoch 1/3
118/118 [==============================] - 91s 771ms/step - loss: 0.6146 - accuracy: 0.6659
Epoch 2/3
118/118 [==============================] - 91s 770ms/step - loss: 0.4891 - accuracy: 0.7651
Epoch 3/3
40/40 [==============================] - 10s 240ms/step - loss: 0.6492 - accuracy: 0.6856
ft_acc- 68.55999827384949


/kaggle/working/model_discovery/utils.py:356: RuntimeWarning: invalid value encountered in true_divide
  prob1 = prob1 / np.sum(prob1)


Already present- Financial_News_1_trained_model
Loaded
8/8 [==============================] - 2s 215ms/step - loss: 0.3462 - accuracy: 0.8780
src_acc- 87.80487775802612
118/118 [==============================] - 29s 248ms/step - loss: 0.8842 - accuracy: 0.6424
ser_acc- 64.24000263214111
Epoch 1/3
118/118 [==============================] - 91s 771ms/step - loss: 0.6286 - accuracy: 0.6595
Epoch 2/3
118/118 [==============================] - 91s 771ms/step - loss: 0.4860 - accuracy: 0.7701
Epoch 3/3
40/40 [==============================] - 10s 240ms/step - loss: 0.6329 - accuracy: 0.7016
ft_acc- 70.16000151634216


/kaggle/working/model_discovery/utils.py:356: RuntimeWarning: invalid value encountered in true_divide
  prob1 = prob1 / np.sum(prob1)


Already present- Financial_News_1_trained_model
Loaded
8/8 [==============================] - 2s 211ms/step - loss: 0.3462 - accuracy: 0.8780
src_acc- 87.80487775802612
106/106 [==============================] - 26s 247ms/step - loss: 0.5269 - accuracy: 0.7672
ser_acc- 76.71721577644348
Epoch 1/3
106/106 [==============================] - 81s 768ms/step - loss: 0.0949 - accuracy: 0.9602
Epoch 2/3
106/106 [==============================] - 81s 768ms/step - loss: 0.0473 - accuracy: 0.9798
Epoch 3/3
36/36 [==============================] - 9s 242ms/step - loss: 0.6276 - accuracy: 0.7128
ft_acc- 71.27564549446106
New model- Tweets_1_trained_model
Epoch 1/3
118/118 [==============================] - 104s 882ms/step - loss: 0.6393 - accuracy: 0.6477 - val_loss: 0.5844 - val_accuracy: 0.6768
Epoch 2/3
118/118 [==============================] - 101s 855ms/step - loss: 0.4991 - accuracy: 0.7536 - val_loss: 0.5985 - val_accuracy: 0.6960
Epoch 3/3
118/118 [==============================] - 101s 8

In [11]:
result_df = pd.DataFrame({'Source': src_list, 'Target': tar_list, 'JSD': jsd_list,'Adaptivity': ada_list, 'L2D':l2d_list,
                         'Source Accuracy': src_acc_list, 'Serving Accuracy': ser_acc_list, 'Finetune Accuracy': ft_acc_list})

In [12]:
result_df=result_df.sort_values(by=['Target','Serving Accuracy'],ascending=False)
result_df.reset_index(drop=True, inplace=True)
result_df.to_csv('Sentiment_serving-train.csv', index=False)
result_df

,Source,Target,JSD,Adaptivity,L2D,Source Accuracy,Serving Accuracy,Finetune Accuracy
0,Financial_News_1,Tweets_hatred_5,0.508087,1.266667,0.153876,87.804878,76.717216,71.275645
1,Financial_News_2,Tweets_hatred_5,0.499096,1.466667,0.153196,88.617885,72.227180,76.984835
2,Movie_2,Tweets_hatred_5,0.114503,15.800000,0.073590,86.879998,66.577458,59.500444
3,Tweets_2,Tweets_hatred_5,0.093903,17.000000,0.052303,71.759999,64.644665,59.322035
4,Movie_1,Tweets_hatred_5,0.113434,16.133333,0.073737,84.320003,59.024680,74.041033
5,Tweets_1,Tweets_hatred_5,0.097748,17.000000,0.052496,68.320000,57.567650,54.326493
6,Tweets_1,Tweets_2,0.006653,17.000000,0.013643,68.320000,70.933336,72.479999
7,Financial_News_1,Tweets_2,0.530934,1.176471,0.170825,87.804878,64.240003,70.160002
8,Financial_News_2,Tweets_2,0.523461,1.352941,0.167609,88.617885,63.546669,68.720001
9,Movie_2,Tweets_2,0.123531,15.470588,0.079556,86.879998,63.226664,69.840002
